# How to export apply code 
<br>
This notebook shows how to export SQL code from a model in hana_ml.
<br>
Two cases are possible:<br>
- One can export apply code immediately after having fitted a model.<br>
- One can export apply code when the model was saved and reloaded later.<br>

In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
# Set PYTHONPATH
# (only required if one wants to directly use the library code sources)
sys.path.append(os.path.dirname('E:/hanamlapi/src/hana_ml'))
sys.path.append('E:/hanamlapi/nutest')

In [3]:
# Import hana_ml library
from hana_ml.dataframe import DataFrame, ConnectionContext
from hana_ml.algorithms.apl.classification import AutoClassifier

In [4]:
# HANA python driver used for export_apply_code function
import pyhdb

In [5]:
# Global variables for Hana connection
from data_load_utils import Settings
HDB_HOST, HDB_PORT, HDB_USER, HDB_PASS, MODEL_SCHEMA = Settings.load_config("E:/hanamlapi/nutest/testscripts/config/e2edata.ini")
HDB_HOST

'10.47.242.42'

# Case 1: export code from a fitted model.
- The model is just fitted. <br>
- User can export apply code right after.

## Creates and trains model

In [6]:
# HANA connection for hana_ml
CONN = ConnectionContext(HDB_HOST, HDB_PORT, HDB_USER, HDB_PASS)

In [7]:
data = DataFrame(CONN, 'select * from APL_SAMPLES.AUTO_CLAIMS_FRAUD')
data.head(5).collect()

,CLAIM_ID,DAYS_TO_REPORT,BODILY_INJURY_AMOUNT,PROPERTY_DAMAGE,PREVIOUS_CLAIMS,PAYMENT_METHOD,IS_REAR_END_COLLISION,PREM_AMOUNT,AGE,GENDER,MARITAL_STATUS,INCOME_ESTIMATE,INCOME_CATEGORY,POLICY_HOLDER,IS_FRAUD
0,CL_0174802,2,0,9025,0,CC,No,Standard,79,Male,Married,8090.4,14,Y,No
1,CL_0856464,0,0,388,0,Auto,No,Safedriving_discount,29,Male,Single,28504.0,25,N,No
2,CL_0789960,3,0,1025,0,CH,Yes,Standard,27,Male,Single,67668.3,50,N,No
3,CL_0534963,28,0,4077,0,CC,No,Standard,37,Male,Married,46812.7,35,N,No
4,CL_0614972,3,0,3199,0,Auto,Yes,Standard,67,Female,Married,26369.2,25,N,No


In [8]:
# Creates model
model = AutoClassifier(CONN)
# Fits model
model.fit(data, key='CLAIM_ID', label='IS_FRAUD')

## Export apply code

In [13]:
'''
Calls APL EXPORT_APPLY_CODE.
Parameters
----------
- model: an instance of hana_ml model.
    The model must be already fitted.
- key: str
    The name of the primary key column
- label: str
    The name of the label (target) column
- schema_name: str
    The schema name of the apply-in table
- table_name: str
    The apply-in table name
- other_params: dict (optional)
    The additional parameters to be included in the configuration.
    That corresponds to additional APL alias user would like to add.

Example
-------
>>> sql = export_apply_code(model=model, 
...               key='CLAIM_ID', label='IS_FRAUD', 
...               schema_name='APL_SAMPLE', table_name='AUTO_CLAIMS_NEW')
>>> print('Exported sql:\n', sql)
'''
def export_apply_code(model, key, label, schema_name, table_name, **other_params):
    # Reuses the same than the model (so the all temporary tables created the model are visible)
    conn = model.conn_context.connection
    cursor = conn.cursor()

    # -- CONFIG table
    try:
        cursor.execute('drop table #EXPORT_APPLY_CODE_CONFIG')
    except:
        pass
    cursor.execute('create local temporary table #EXPORT_APPLY_CODE_CONFIG (KEY NVARCHAR(1000), VALUE NCLOB, CONTEXT NVARCHAR(100))')
    # Add parameters into CONFIG table
    # Default params
    params = {
        'APL/CodeType': 'HANA',
        'APL/CodeKey': key,
        'APL/CodeTarget': label,
        'APL/CodeSpace': '"{}"."{}"'.format(schema_name, table_name),
        'APL/ApplyExtraMode':'Advanced Apply Settings',
        'APL/ApplyDecision':'true',
        'APL/ApplyProbaDecision':'true',
        'APL/CodeUseVarNameAlias':'true',
    }
    # Additional parameters to be put in CONFIG table
    for k in other_params:
         params[k] = other_params[k]
    # isnert params to CONFIG table
    cursor.executemany('insert into #EXPORT_APPLY_CODE_CONFIG values (?, ?, NULL)', [(k, v) for k, v in params.items()])

    # -- Output table
    try:
        cursor.execute('drop table #EXPORT_APPLY_CODE_OUTPUT')
    except:
        pass
    cursor.execute('create local temporary table #EXPORT_APPLY_CODE_OUTPUT (OID NVARCHAR(50), KEY NVARCHAR(100), VALUE NCLOB)')

    # -- HEADER_FUNC
    try:
        cursor.execute('drop table #EXPORT_APPLY_CODE_HEADER')
    except:
        pass
    cursor.execute('create local temporary table #EXPORT_APPLY_CODE_HEADER (KEY NVARCHAR(50), VALUE NVARCHAR(255))')

    # Call APL
    sql = 'call "_SYS_AFL"."APL_EXPORT_APPLY_CODE" (#EXPORT_APPLY_CODE_HEADER, {model_table}, #EXPORT_APPLY_CODE_CONFIG, #EXPORT_APPLY_CODE_OUTPUT) with overview'
    model_table_name = model.model_table_.name  # the temp table where the model is saved
    sql = sql.format(model_table=model_table_name)
    cursor.execute(sql)
    
    # Retrieves the sql generated
    cursor.execute('select to_char(VALUE) value from #EXPORT_APPLY_CODE_OUTPUT')
    sql = cursor.fetchone()[0]
    return sql

## Calls export_apply_code 

In [14]:
# export_apply_code
sql = export_apply_code(model=model, 
                  key='CLAIM_ID', label='IS_FRAUD', 
                  schema_name='APL_SAMPLE', table_name='AUTO_CLAIMS_NEW')
print('Exported sql:\n', sql)

Exported sql:
 -- Automated Analytics - Unlicensed Version 10.1905.0.0 - Copyright 2018 SAP SE or an SAP affiliate company. All rights reserved. - Model built in 9.3.0.0 - Model Name is APLModel - Model Version is 1
SELECT CLAIM_ID, "rr_IS_FRAUD", (CASE WHEN "rr_IS_FRAUD" >= 2.81733766601e-1 THEN 'Yes'
ELSE 'No'
END)
AS "decision_rr_IS_FRAUD", CAST((CASE 
WHEN "rr_IS_FRAUD" < -1.461836597006e0 THEN 1.0e0
WHEN "rr_IS_FRAUD" <= 3.716341586901e-2 THEN 1.0e0
WHEN "rr_IS_FRAUD" <= 3.81634029944e-2 THEN  ( -1.351368749689e-2*"rr_IS_FRAUD"+1.000502214788e0 ) 
WHEN "rr_IS_FRAUD" <= 6.343447913304e-2 THEN  ( -5.34742305369e-4*"rr_IS_FRAUD"+1.000006894073e0 ) 
WHEN "rr_IS_FRAUD" <= 6.345977550555e-2 THEN  ( -5.336733554998e2*"rr_IS_FRAUD"+3.485326430628e1 ) 
WHEN "rr_IS_FRAUD" <= 7.712594791276e-2 THEN  ( -9.878406036262e-1*"rr_IS_FRAUD"+1.049161115914e0 ) 
WHEN "rr_IS_FRAUD" <= 1.037060485469e-1 THEN  ( -2.611364056357e-1*"rr_IS_FRAUD"+9.931133657922e-1 ) 
WHEN "rr_IS_FRAUD" <= 1.422911137913e-

# Case 2: export code from a reloaded model.
- The model was trained and saved. <br>
- Now, we can reload the model and export apply code.

## Saves and reloads a model

In [15]:
from hana_ml.model_storage import ModelStorage
from hana_ml.model_storage import ModelStorageError

# Creates an object model_storage
MODEL_SCHEMA='MODEL_STORAGE'  # schema name used to store the model data
# model storage must use the same connection than the model
model_storage = ModelStorage(connection_context=model.conn_context,
                             schema=MODEL_SCHEMA)

In [16]:
# Saves the model we had trained previously
# The new model is saved with an unique name
model.name = 'My AUTO_CLAIMS model'

model_storage.save_model(model=model, if_exists='replace')
model_storage.list_models()

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,MODEL_STORAGE_VER
0,AutoClassifier 1,1,APL,hana_ml.algorithms.apl.classification.AutoClas...,"{""model_attributes"": {""name"": ""AutoClassifier ...",2019-08-27 11:02:57,1
1,My K2R IRIS model,1,APL,hana_ml.algorithms.apl.classification.AutoClas...,"{""model_attributes"": {""name"": ""My K2R IRIS mod...",2019-08-27 11:07:17,1
2,My Census model,1,APL,hana_ml.algorithms.apl.classification.AutoClas...,"{""model_attributes"": {""name"": ""My Census model...",2019-11-29 10:05:12,1
3,My AUTO_CLAIMS model,1,APL,hana_ml.algorithms.apl.classification.AutoClas...,"{""model_attributes"": {""name"": ""My AUTO_CLAIMS ...",2019-11-29 13:34:34,1


## Reloads the saved model

In [17]:
# Close the session
CONN.close()
# Reopens connection
CONN = ConnectionContext(HDB_HOST, HDB_PORT, HDB_USER, HDB_PASS)
MODEL_SCHEMA='MODEL_STORAGE'  # schema name used to store the model data

# Reloads the model
model_storage = ModelStorage(connection_context=CONN, schema=MODEL_SCHEMA)
model2 = model_storage.load_model(name='My Census model')

mod_name: hana_ml.algorithms.apl.classification


## Export code

In [18]:
# Example
sql = export_apply_code(model=model2, 
                  key='CLAIM_ID', label='IS_FRAUD', 
                  schema_name='APL_SAMPLE', table_name='AUTO_CLAIMS_NEW')
print('Exported sql:\n', sql)

Exported sql:
 -- Automated Analytics - Unlicensed Version 10.1905.0.0 - Copyright 2018 SAP SE or an SAP affiliate company. All rights reserved. - Model built in 9.3.0.0 - Model Name is APLModel - Model Version is 1
SELECT CLAIM_ID, "rr_class", (CASE WHEN "rr_class" >= 2.541732350001e-1 THEN 1
ELSE 0
END)
AS "decision_rr_class", CAST((CASE 
WHEN "rr_class" < -1.87592389104e0 THEN 9.993761696818e-1
WHEN "rr_class" <= -3.769238781658e-1 THEN 9.993761696818e-1
WHEN "rr_class" <= -3.759238910404e-1 THEN  ( -3.112782451028e-3*"rr_class"+9.982028876485e-1 ) 
WHEN "rr_class" <= -1.513137164263e-1 THEN  ( -1.385842106408e-5*"rr_class"+9.993678472279e-1 ) 
WHEN "rr_class" <= -1.510888814167e-1 THEN  ( -1.383071808078e1*"rr_class"-1.09340740945e0 ) 
WHEN "rr_class" <= -1.375073674063e-1 THEN  ( -2.289604553961e-1*"rr_class"+9.616669354698e-1 ) 
WHEN "rr_class" <= -1.106982897973e-1 THEN  ( -5.876153177418e-2*"rr_class"+9.850705413925e-1 ) 
WHEN "rr_class" <= -9.926380045382e-2 THEN  ( -1.3777112